In [1]:
BASEURL="https://replicate.com/meta/llama-2-13b-chat"
import selenium
path="C:\Program Files (x86)\msedgedriver.exe"
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import os
import re
import csv
import replicate
REPLICATE_API_TOKEN = "r8_OUIZ8VqW8IORJ9snizKgPggxdBr9d4f1n7Wtb"
os.environ["REPLICATE_API_TOKEN"] = REPLICATE_API_TOKEN

<>:3: SyntaxWarning: invalid escape sequence '\P'
<>:3: SyntaxWarning: invalid escape sequence '\P'
C:\Users\9616k\AppData\Local\Temp\ipykernel_4508\3246455475.py:3: SyntaxWarning: invalid escape sequence '\P'
  path="C:\Program Files (x86)\msedgedriver.exe"


In [2]:
INPUT_DIR="CATEGORICAL_FORMATTED_VIKASPEDIA/Crop-Production/"
OUTPUT_DIR="output/Crop-Production/"
if not (os.path.exists(OUTPUT_DIR)):
    os.mkdir(OUTPUT_DIR)

In [3]:
def writeFile(dict,output_path="output.csv"):
    fields = ["question", "answer"]
    with open(output_path, 'a+', newline='',encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fields)
        if csvfile.tell() == 0:  # Check if the file is empty (header not written)
            writer.writeheader()
        for key, value in dict.items():
            writer.writerow({'question': key.replace('\n','\\n'), 'answer': value.replace('\n','\\n')})


def writeQuestionsFile(temp,output_path="output.txt"):
    with open(output_path, 'a+', newline='') as textFile:
        for que in temp:
            textFile.write(que+"\n\n")
    print("QUE GENERATED IN: ",output_path)

def readCSV(input_path="input.csv"):
    with open(input_path, 'r', newline='', encoding='utf-8') as csvfile:
        reader = csv.reader(csvfile)
        header = next(reader)  # Read the header

        # Print or handle the header as needed
        print(f"Header: {header}")

        # Iterate through rows
        for row in reader:
            # Process the row and handle escape sequences as needed
            processed_row = []
            for item in row:
                processed_row.append(item.replace("\\n", "\n"))  # Replace '\\n' with actual newline character
            answer = processed_row[1]  # Assuming the second column is the 'answer'
            answerRows = answer.split('\n')  # Split the answer by newline
            print(f"{processed_row[0]}: {answerRows}\n")

In [4]:

def process_content(prompt,context,driver,generatingQuestions=False):
    try:
        textarea_prompt = driver.find_element(By.ID, 'prompt')
        textarea_prompt.clear()
        textarea_prompt.send_keys(prompt)
        if not generatingQuestions:
            textarea_context = driver.find_element(By.ID, 'system_prompt')
            textarea_context.clear()
            textarea_context.send_keys(context)
            max_new_tokens = driver.find_element(By.ID, 'max_new_tokens')
            max_new_tokens.clear()
            new_value = 6000  # Replace with your desired numeric value
            max_new_tokens.send_keys(str(new_value))
            temp = driver.find_element(By.ID, 'temperature')
            temp.clear()
            new_value = 0.01  # Replace with your desired numeric value
            temp.send_keys(str(new_value))
            seed = driver.find_element(By.ID, 'seed')
            seed.clear()
            new_value = 1  # Replace with your desired numeric value
            seed.send_keys(str(new_value))
        form = driver.find_element(By.ID, 'input-form')  # Replace with the actual form locator
        form.submit()
        time.sleep(30)
        # Fetch content of div with class 'output' and print
        output_div = driver.find_element(By.CLASS_NAME, 'output')
        output_text = output_div.text
        # print(output_text)
        return output_text
    except:
        return ""

In [5]:
def processFile(context,driver,FILENAME):
    prompt=r"Generate all possible questions related to the context provided in the following format only:1.question1 \n 2.question2 \n 3.question3,..],where \n means line break, provide serial number as given, nothing extra is to be added"
    inputTokenLimit=4050
    s=""
    queAns={}
    try:
        s=process_content(prompt,context,driver)
        # filtered_str = re.sub('^Sure.*\n', '', s.strip())
        lines = s.split('\n')
        temp = [line.strip()[2:] for line in lines if line.strip() != '']
        temp=temp[1:]
        writeQuestionsFile(temp,OUTPUT_DIR+FILENAME)
        prompt="?Generate answer with respect to the context provided"
        for line in temp:
            queAns[line]=process_content(line+prompt,context,driver,True)
        return queAns
    except Exception as e:
        print("An error occurred:", e)
        return {}

In [6]:
def generateResponse(context,FILENAME):
    driver = webdriver.Edge()
    driver.get(BASEURL)
    temp=processFile(context,driver,FILENAME)
    driver.quit()
    return temp

In [7]:
for iteration,i in enumerate(os.listdir(INPUT_DIR)):
    print(iteration,i)

0 Abroma-augusta-.txt
1 Acid-lime-crop-in-foothills-of--Arunachal-Pradesh.txt
2 Aconitum-balfourii-.txt
3 Aconitum-heterophyllum.txt
4 Aegle-marmelos.txt
5 AESA-based-IPM.txt
6 Agro-climatic-zones-in-India.txt
7 Allelobiosis-.txt
8 Alpinia-galanga.txt
9 Alstonia-scholaris.txt
10 Alternate-bearing-in-Apple-and-its-management.txt
11 Alternate-bearing-in-mango-and-its-management.txt
12 Amla-Beneficial-insects.txt
13 Amla-Crop-Stage-Wise-IPM.txt
14 Amla-Diseases-.txt
15 Amla-Insect-Pests.txt
16 Amla-pests.txt
17 Amla.txt
18 Andrographis-paniculata.txt
19 Angelica-glauca.txt
20 Aonla-in-foothills-of-Arunachal-Pradesh.txt
21 Apple--Crop-Stage-wise-IPM.txt
22 Apple--Diseases-and-Symptoms.txt
23 Apple--Natural-Enemies.txt
24 Apple-Insect-and-Mite-Management.txt
25 Apple-Pests-.txt
26 Apple-production-techniques.txt
27 Applications-of-Super-Absorbent-Polymers-in-Agriculture.txt
28 Apricot-Beneficial-Insects.txt
29 Apricot-Crop-Stage-Wise-IPM.txt
30 Apricot-Diseases.txt
31 Apricot-Insect-Pests.t

In [ ]:
import os
res=[]
for iteration,i in enumerate(os.listdir(INPUT_DIR)[0:1]):
    context = open(INPUT_DIR+i, "r",encoding="utf-8").read()
    context=re.sub('\n','',context)
    context=context[:7000]
    
    response=generateResponse(context,i)
    
    writeFile(response,OUTPUT_DIR+i[:len(i)-4]+".csv")
    res.append(response)
    
    if len(response)>1:
    # writeFile(response,"New/latest/"+i)
        print("DONE FOR",iteration,":",i[:len(i)-4])
    else:
        print("FAILED FOR",iteration,":",i[:len(i)-4])
        